The aim of this is to build a recommendation algorithm that will take a Marvel Cinematic Universe Film and return comic book recommendations based on the characters within that movie. 

In [1]:
import pandas as pd
import numpy as np

characters = pd.read_csv('characters.csv')
comics = pd.read_csv('comics.csv')
mapper = pd.read_csv('charactersToComics.csv')

In [2]:
comics.head()

,comicID,title,issueNumber,description
0,16232,Cap Transport (2005) #12,12.0,NaN
1,16248,Cap Transport (2005) #9,9.0,NaN
2,4990,Halo Preview (2006),0.0,NaN
3,21486,Ultimate X-Men (Spanish Language Edition) (200...,9.0,NaN
4,58634,A Year of Marvels: The Incredible (2016) #5,5.0,It’s Halloween in the Marvel U! What does that...


In [3]:
comics = comics[~comics.title.str.contains('Zombie')]

In [4]:
characters.head()

,characterID,name
0,1009220,Captain America
1,1010740,Winter Soldier
2,1009471,Nick Fury
3,1009552,S.H.I.E.L.D.
4,1009228,Sharon Carter


In [5]:
mapper.head()

,comicID,characterID
0,16232,1009220
1,16232,1010740
2,16248,1009220
3,16248,1009471
4,16248,1009552


In [6]:
comics.describe()

,comicID,issueNumber
count,41078.000000,41078.000000
mean,30827.915380,53.855046
std,20094.238412,165.217879
min,2.000000,-1.000000
25%,13144.250000,1.000000
50%,26915.500000,7.000000
75%,49261.750000,36.000000
max,73980.000000,16584.000000


In [7]:
characters.describe()

,characterID
count,1.170000e+03
mean,1.010507e+06
std,1.289773e+03
min,1.009144e+06
25%,1.009495e+06
50%,1.010724e+06
75%,1.011086e+06
max,1.017583e+06


I will be using https://www.cbr.com/mcu-all-marvel-characters-movies/ to identify all of the characters used in the movies. This article was published in 2021, so I will be updating a few to account for any content that has come out since then.

In [8]:
mcu_character_appearances = {
    'Dormammu': ['Doctor Strange'],
    'Ronan': ['Guardians of the Galaxy','Captain Marvel'],
    'Crossbones':['Captain America: The Winter Soldier','Captain America: Civil War','Avengers: Endgame'],
    'Zemo':['Captain America: Civil War','The Falcon and The Winter Soldier'],
    "Baron Zemo (Heinrich Zemo)":['Captain America: Civil War','The Falcon and The Winter Soldier'],
    "Baron Zemo (Helmut Zemo)":['Captain America: Civil War','The Falcon and The Winter Soldier'],
    'Arnim Zola':['Captain America: The First Avenger','Captain America: The Winter Soldier'],
    'Wong':['Doctor Strange','Doctor Strange in the Multiverse of Madness','Spider-Man: No Way Home','Shang-Chi','She-Hulk','Avengers: Infinity War','Avengers: Endgame'],
    'Maria Hill':['The Avengers','Captain America: The Winter Soldier','Avengers: Age of Ultron','Avengers: Infinity War'],
    'Justin Hammer':['Iron Man 2'],
    'Ultron':['Avengers: Age of Ultron'],
    'Mantis':['Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Red Skull':['Captain America: The First Avenger'],
    'Abomination (Emil Blonsky)' : ['The Incredible Hulk'],
    'Thaddeus Ross' : ['The Incredible Hulk','Captain America: Civil War','Avengers: Infinity War'],
    'Vulture (Adrian Toomes)' : ['Spider-Man: Homecoming'],
    'Mysterio':['Spider-Man: Far From Home'],
    'Vision': ['Avengers: Age of Ultron','Captain America: Civil War','Avengers: Infinity War', 'WandaVision'],
    'Odin':['Thor','Thor: The Dark World','Thor: Ragnarok'],
    'Groot': ['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Nebula':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War','Avengers: Endgame'],
    'Scarlet Witch':['Avengers: Age of Ultron','Captain America: Civil War','Avengers: Infinity War', 'WandaVision', 'Doctor Strange in the Multiverse of Madness'],
    'Wiccan':['WandaVision', 'Doctor Strange in the Multiverse of Madness'],
    'Speed':['WandaVision', 'Doctor Strange in the Multiverse of Madness'],
    'Falcon':['Captain America: The Winter Soldier','Captain America: Civil War','Avengers: Infinity War','The Falcon and The Winter Soldier'],
    'Guardians of the Galaxy':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Avengers': ['The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame','Captain America: Civil War'],
    'Drax':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Thanos':['Avengers: Infinity War','Avengers: Endgame'],
    'Jane Foster':['Thor','Thor: The Dark World','Thor: Love and Thunder'],
    'Thor (Goddess of Thunder)': ['Thor: Love and Thunder'],
    'Winter Soldier': ['Captain America: The First Avenger', "Captain America: The Winter Soldier", 'Captain America: Civil War', 'Avengers: Infinity War','The Falcon and The Winter Soldier'],
    'Patriot':['The Falcon and The Winter Soldier'],
    'Hank Pym': ['Ant Man', 'Ant Man and the Wasp'],
    'Ant-Man (Scott Lang)': ['Ant Man', 'Ant Man and the Wasp', 'Captain America: Civil War', 'Avengers: Endgame'],
    'Hawkeye': ['The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame','Captain America: Civil War', 'Hawkeye'],
    "Clint Barton": ['The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame','Captain America: Civil War', 'Hawkeye'],
    'Hawkeye (Kate Bishop)':['Hawkeye'],
    'Wasp': ['Ant Man', 'Ant Man and the Wasp'],
    'Rhodey':['Iron Man','Iron Man 2','Iron Man 3','Captain America: Civil War', 'Avengers: Infinity War'],
    "T'Challa" :['Black Panther','Captain America: Civil War', 'Avengers: Infinity War'],
    "Black Panther":['Black Panther','Captain America: Civil War', 'Avengers: Infinity War'],
    "Pepper Potts":['Iron Man', 'Iron Man 2','Iron Man 3', 'Avengers: Endgame'],
    'Captain Marvel (Carol Danvers)':['Captain Marvel','Avengers: Endgame'],
    'Carol Danvers':['Captain Marvel','Avengers: Endgame'],
    'Captain America': ['Captain America: The First Avenger','Captain America: The Winter Soldier','Captain America: Civil War', 'The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame'],
    "Steve Rogers": ['Captain America: The First Avenger','Captain America: The Winter Soldier','Captain America: Civil War', 'The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame'],
    'Captain America (Sam Wilson)': ['Captain America: The Winter Soldier','Captain America: Civil War','Avengers: Infinity War','The Falcon and The Winter Soldier'],
    'Gamora':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Rocket Raccoon':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Star-Lord (Peter Quill)':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Peter Quill':['Guardians of the Galaxy','Guardians of the Galaxy Vol. 2','Avengers: Infinity War'],
    'Loki':['Thor','The Avengers','Thor: The Dark World','Thor: Ragnarok','Loki'],
    'Doctor Strange':['Doctor Strange','Doctor Strange in the Multiverse of Madness','Spider-Man: No Way Home','Avengers: Infinity War'],
    'Stephen Strange':['Doctor Strange','Doctor Strange in the Multiverse of Madness','Spider-Man: No Way Home','Avengers: Infinity War'],
    "Black Widow": ['Iron Man 2', 'The Avengers', "Captain America: The Winter Soldier","Avengers: Age of Ultron","Avengers: Infinity War",'Avengers: Endgame',"Black Widow"],
    "Natasha Romanoff": ['Iron Man 2', 'The Avengers', "Captain America: The Winter Soldier","Avengers: Age of Ultron","Avengers: Infinity War",'Avengers: Endgame',"Black Widow"],
    "Bruce Banner":['The Incredible Hulk','The Avengers','Avengers: Age of Ultron','Thor: Ragnarok','Avengers: Infinity War', 'Avengers: Endgame','She-Hulk'],
    "Hulk":['The Incredible Hulk','The Avengers','Avengers: Age of Ultron','Thor: Ragnarok','Avengers: Infinity War', 'Avengers: Endgame','She-Hulk'],
    "She-Hulk (Jennifer Walters)":['She-Hulk'],
    "Peter Parker":['Spider-Man: Homecoming','Spider-Man: Far From Home','Spider-Man: No Way Home','Captain America: Civil War'],
    "Spider-Man":['Spider-Man: Homecoming','Spider-Man: Far From Home','Spider-Man: No Way Home','Captain America: Civil War'],
    "Thor":['Thor','Thor: The Dark World','Thor: Ragnarok', "Thor: Love and Thunder",'The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame','Captain America: Civil War'],
    "Young Avengers":['WandaVision','Hawkeye','The Falcon and The Winter Soldier', 'Doctor Strange in the Multiverse of Madness'],
    "Iron Man": ['Iron Man','Iron Man 2',"Iron Man 3",'The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame','Captain America: Civil War','Spider-Man: Homecoming'],
    "Tony Stark":['Iron Man','Iron Man 2',"Iron Man 3",'The Avengers','Avengers: Age of Ultron','Avengers: Infinity War','Avengers: Endgame','Captain America: Civil War','Spider-Man: Homecoming'],
    "Ms. Marvel (Kamala Khan)": ['Ms. Marvel'],
    "Moon Knight":["Moon Knight"],
    "Shang-Chi":['Shang-Chi'],
    "Sharon Carter":['Captain America: The Winter Soldier','Captain America: Civil War','The Falcon and The Winter Soldier'],
    "Lizard":['Spider-Man: No Way Home'],
    "Ego":['Guardians of the Galaxy Vol. 2'],
    "J. Jonah Jameson": ['Spider-Man: No Way Home'],
    "A.I.M.":["Iron Man 3"],
    "Batroc the Leaper":['Captain America: The Winter Soldier',"The Falcon and The Winter Soldier"],
    "Iron Patriot (James Rhodes)":['Iron Man 2','Iron Man 3', 'Captain America: Civil War','Avengers: Infinity War'],
    "U.S. Agent":['The Falcon and The Winter Soldier'],
    "Sif":['Thor',"Thor: The Dark World","Thor: Love and Thunder","Loki"],
    "Kang":['Loki'],
    "Miss America":['Doctor Strange in the Multiverse of Madness'],
    "Shocker (Herman Schultz)": ['Spider-Man: No Way Home'],
    "Curt Conners":['Spider-Man: No Way Home'],
    "Dum Dum Dugan":['Captain America: The First Avenger'],
    "S.H.I.E.L.D.": ['Captain America: The Winter Soldier','The Avengers'],
    "Eternals":["The Eternals"],
    "Enchantress (Sylvie Lushton)":["Loki"],
    "The Howling Commandos":['Captain America: The First Avenger'],
    "Obadiah Stane":['Iron Man'],
    "Kate Bishop":['Hawkeye'],

}

In [9]:
mcu_years = {'Spider-Man: No Way Home':2021, 'Avengers: Age of Ultron': 2015,
       'Ant Man and the Wasp':2018, 'WandaVision':2020, 'Thor: Love and Thunder':2022,
       'Ant Man':2015, 'Thor: The Dark World':2013, 'Spider-Man: Homecoming':2017, 'Loki':2020,
       'The Avengers':2012, 'Avengers: Endgame':2019, 'The Eternals':2021, 'Iron Man':2008,
       'Captain America: Civil War':2016, 'Avengers: Infinity War':2018,
       'Spider-Man: Far From Home':2019, 'Doctor Strange':2016,
       'The Falcon and The Winter Soldier': 2020, 'Spider-Man: Homecoming':2017,
       'Shang-Chi':2021, 'Black Widow':2021,
       'Doctor Strange in the Multiverse of Madness':2022,
       'Guardians of the Galaxy':2014, 'Thor: Ragnarok':2017, 'She-Hulk':2022,
       'Black Panther':2018, 'The Incredible Hulk':2010,
       'Captain America: The First Avenger':2011, 'Thor':2011, 'Moon Knight':2022,
       'Iron Man 3':2013, 'Guardians of the Galaxy Vol. 2':2017, 'Hawkeye':2021,
       'Captain America: The Winter Soldier':2013, 'Captain Marvel':2019, 'Iron Man 2':2010,
       'Ms. Marvel':2020}

In [10]:
mcu_char = list(mcu_character_appearances.keys())
movies = list(set(np.concatenate([x for x in mcu_character_appearances.values()])))
mcu_df = pd.DataFrame(columns=movies, index=mcu_char)

for i,row in mcu_df.iterrows():
    for movie in movies:
        mcu_df.loc[i, movie] = True if movie in mcu_character_appearances[i] else False
mcu_df = mcu_df.T

In [11]:

mcu_df.head()

,Dormammu,Ronan,Crossbones,Zemo,Baron Zemo (Heinrich Zemo),Baron Zemo (Helmut Zemo),Arnim Zola,Wong,Maria Hill,Justin Hammer,...,Miss America,Shocker (Herman Schultz),Curt Conners,Dum Dum Dugan,S.H.I.E.L.D.,Eternals,Enchantress (Sylvie Lushton),The Howling Commandos,Obadiah Stane,Kate Bishop
Avengers: Infinity War,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
Doctor Strange,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
Ant Man,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ant Man and the Wasp,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Iron Man 3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
mcu_df['year'] = mcu_df.index.map(mcu_years)

# Truncate Character DF

In [13]:
characters.head()

,characterID,name
0,1009220,Captain America
1,1010740,Winter Soldier
2,1009471,Nick Fury
3,1009552,S.H.I.E.L.D.
4,1009228,Sharon Carter


In [14]:
characters = characters[characters.name.isin(mcu_char)]
characters.head()

,characterID,name
0,1009220,Captain America
1,1010740,Winter Soldier
3,1009552,S.H.I.E.L.D.
4,1009228,Sharon Carter
6,1010808,Hawkeye (Kate Bishop)


# Clean Mapper

In [15]:
mapper = mapper[mapper.characterID.isin(characters.characterID)]
mapper.head()

,comicID,characterID
0,16232,1009220
1,16232,1010740
2,16248,1009220
4,16248,1009552
5,16248,1009228


# Clean Comics

In [16]:
comics = comics[comics.comicID.isin(mapper.comicID)]
comics.head()

,comicID,title,issueNumber,description
0,16232,Cap Transport (2005) #12,12.0,NaN
1,16248,Cap Transport (2005) #9,9.0,NaN
4,58634,A Year of Marvels: The Incredible (2016) #5,5.0,It’s Halloween in the Marvel U! What does that...
7,16241,Cap Transport (2005) #20,20.0,NaN
15,16234,Cap Transport (2005) #14,14.0,NaN


In [17]:
comics['series'] = comics.title.str.split("#", expand=True)[0]
comics['year'] = comics.title.str.split("(", expand=True)[1]
comics['year'] = comics.year.str.split(")", expand=True)[0]

In [18]:
comics['series'].value_counts()

The Amazing Spider-Man (1963)                                  435
Avengers (1963)                                                397
Thor (1966)                                                    375
Incredible Hulk (1962)                                         362
Amazing Spider-Man (1999)                                      343
                                                              ... 
What If? Classic Vol. 5 (Trade Paperback)                        1
Onslaught Reborn (Trade Paperback)                               1
X-Men: The Complete Onslaught Epic Book 4 (Trade Paperback)      1
Avengers Epic Collection: Masters of Evil (Trade Paperback)      1
Wolverine (2013)                                                 1
Name: series, Length: 2225, dtype: int64

In [19]:
sorted_years = sorted(comics['year'].unique(),reverse=True)
sorted_years

['Variant',
 'Trade Paperback',
 'Thor the Mighty Avenger',
 'Spider-Man',
 'Secret Empire',
 'New Printing',
 'New ',
 'MDCU',
 'John Cassaday Cover',
 'Issues 615-619',
 'Iron Man',
 'Hardcover',
 'Graphic Novel',
 'GRAPHIC NOVEL',
 'Digest',
 'Daredevil Vol. I: Guardian Devil',
 'DM Only',
 'Captain America',
 'Book Market',
 'Archie',
 'All-New Guardians of the Galaxy',
 'All-New Edition',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1st',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994',
 '1993',
 '1992',
 '1991',
 '1990',
 '1989',
 '1988',
 '1987',
 '1986',
 '1985',
 '1984',
 '1983',
 '1982',
 '1981',
 '1980',
 '1979',
 '1978',
 '1977',
 '1976',
 '1975',
 '1974',
 '1973',
 '1972',
 '1971',
 '1970',
 '1968',
 '1967',
 '1966',
 '1965',
 '1964',
 '1963',
 '1962',
 '1961',
 '1959',
 '1952',
 '1951',
 '1941',
 '1940',
 '1939']

In [20]:
len(comics['year'].unique())/2

43.0

In [21]:
sorted_years[43]

'1998'

In [35]:
#Convert the strange text ones to the median year
unwanted_strings = ['Variant',
 'Trade Paperback',
 'Thor the Mighty Avenger',
 'Spider-Man',
 'Secret Empire',
 'New Printing',
 'New ',
 'MDCU',
 'John Cassaday Cover',
 'Issues 615-619',
 'Iron Man',
 'Hardcover',
 'Graphic Novel',
 'GRAPHIC NOVEL',
 'Digest',
 'Daredevil Vol. I: Guardian Devil',
 'DM Only',
 'Captain America',
 'Book Market',
 'Archie',
 'All-New Guardians of the Galaxy',
 'All-New Edition',
 '1st']

comics.loc[comics.year.isin(unwanted_strings)]

,comicID,title,issueNumber,description,series,year
18305,1237,Marvel Masterworks: Daredevil Vol. II - 2nd Ed...,0.0,NaN,Marvel Masterworks: Daredevil Vol. II - 2nd Ed...,1st
18356,1243,Marvel Masterworks: The Avengers Vol. III - 2n...,0.0,NaN,Marvel Masterworks: The Avengers Vol. III - 2n...,1st
18482,1211,Marvel Masterworks: Daredevil Vol. I - 2nd Edi...,0.0,NaN,Marvel Masterworks: Daredevil Vol. I - 2nd Edi...,1st
18498,1208,Marvel Masterworks: The Amazing Spider-Man Vol...,0.0,NaN,Marvel Masterworks: The Amazing Spider-Man Vol...,1st
18605,1213,Marvel Masterworks: The Avengers Vol. II - 2nd...,0.0,NaN,Marvel Masterworks: The Avengers Vol. II - 2nd...,1st


In [36]:
comics.loc[comics.year.isin(unwanted_strings),'year']= 1998

In [37]:
comics.year.value_counts()

1998    1087
1963     881
2010     740
1968     685
1999     538
        ... 
1979       7
1978       4
1981       2
1973       2
1965       1
Name: year, Length: 64, dtype: int64

In [38]:
comics.year = comics.year.astype(int)

In [39]:
comics.head()

,comicID,title,issueNumber,description,series,year
0,16232,Cap Transport (2005) #12,12.0,NaN,Cap Transport (2005),2005
1,16248,Cap Transport (2005) #9,9.0,NaN,Cap Transport (2005),2005
4,58634,A Year of Marvels: The Incredible (2016) #5,5.0,It’s Halloween in the Marvel U! What does that...,A Year of Marvels: The Incredible (2016),2016
7,16241,Cap Transport (2005) #20,20.0,NaN,Cap Transport (2005),2005
15,16234,Cap Transport (2005) #14,14.0,NaN,Cap Transport (2005),2005


# Merge Comics Data

In [40]:
comic_books = mapper.merge(characters, on='characterID')
comic_books = comic_books.merge(comics, on ='comicID')
comic_books.head()

,comicID,characterID,name,title,issueNumber,description,series,year
0,16232,1009220,Captain America,Cap Transport (2005) #12,12.0,NaN,Cap Transport (2005),2005
1,16232,1010740,Winter Soldier,Cap Transport (2005) #12,12.0,NaN,Cap Transport (2005),2005
2,16248,1009220,Captain America,Cap Transport (2005) #9,9.0,NaN,Cap Transport (2005),2005
3,16248,1009552,S.H.I.E.L.D.,Cap Transport (2005) #9,9.0,NaN,Cap Transport (2005),2005
4,16248,1009228,Sharon Carter,Cap Transport (2005) #9,9.0,NaN,Cap Transport (2005),2005


In [41]:
comic_books['character_present'] = 1

In [55]:
comic_books_pivot = comic_books.pivot_table(index='series', columns='name', values='character_present').fillna(0)

In [56]:
comic_books_pivot = comic_books_pivot.merge(comic_books[['series','year']], on='series')

In [60]:
comic_books_pivot = comic_books_pivot.groupby('series').max()

In [46]:
# get the columns in the right order here
mcu_df = mcu_df[comic_books_pivot.columns]

In [67]:
from sklearn.neighbors import NearestNeighbors

X = comic_books_pivot.drop(columns='year').copy()
n = NearestNeighbors(n_neighbors=5)
n.fit(X)

NearestNeighbors()

In [68]:
def recommend_comics(movie):
    movie_data = mcu_df.drop(columns='year').loc[movie].values.reshape(1,-1)
    rec_index = n.kneighbors(movie_data)[1][0]
    return comic_books_pivot.iloc[rec_index].index

    

In [69]:
recommend_comics('Captain America: The Winter Soldier')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Cap Transport (2005) ',
       'Captain America: The Death of Captain America Vol. 2 - The Burden of Dreams (Trade Paperback)',
       'Captain America: Winter Soldier Vol. 2 (Trade Paperback)',
       'Captain America: Reborn MGC (2011) ',
       'Captain America and the Secret Avengers (2011) '],
      dtype='object', name='series')

In [70]:
recommend_comics('Thor: Love and Thunder')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Mighty Thor Vol. 3: The Asgard/Shi'ar War (Trade Paperback)',
       'Devil Dinosaur by Jack Kirby Omnibus (Hardcover)',
       'Mighty Thor Vol. 4: The War Thor (Hardcover)',
       'Marvel Masterworks: The Mighty Thor Vol. 5 (Hardcover)',
       'Generations: The Unworthy Thor & The Mighty Thor (2017) '],
      dtype='object', name='series')

In [71]:
recommend_comics('Captain Marvel')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Captain Marvel: Earth's Mightiest Hero Vol. 3 (Trade Paperback)',
       'Captain Marvel: Carol Danvers - The Ms. Marvel Years Vol. 2 (Trade Paperback)',
       'Captain Marvel (2012) ',
       'Captain Marvel: Earth's Mightiest Hero Vol. 4 (Trade Paperback)',
       'Captain Marvel (2014) '],
      dtype='object', name='series')

In [72]:
recommend_comics('WandaVision')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Young Avengers by Allan Heinberg & Jim Cheung: The Children's Crusade (Trade Paperback)',
       'Vision (2015) ', 'Vision and the Scarlet Witch (1985) ',
       'Avengers West Coast: Vision Quest (Trade Paperback)',
       'Avengers Origins: Scarlet Witch & Quicksilver (2011) '],
      dtype='object', name='series')

In [73]:
recommend_comics('Avengers: Age of Ultron')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Marvel Universe Avengers Assemble: Civil War (2017) ',
       'Marvel Universe Avengers: Ultron Revolution Vol. 3 (Digest)',
       'Avengers Legends Vol. 2: The Korvac Saga (Trade Paperback)',
       'Avengers: Galactic Storm Vol. 2 (Trade Paperback)',
       'Avengers: The Serpent Crown (Trade Paperback)'],
      dtype='object', name='series')

In [74]:
recommend_comics('Captain America: Civil War')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Captain America (1968) ',
       'Marvel Masterworks: Captain America Vol. 4 (Hardcover)',
       'Secret Empire (2017) ',
       'Avengers: The Serpent Crown (Trade Paperback)',
       'Avengers/Defenders War Premiere (Hardcover)'],
      dtype='object', name='series')

In [75]:
recommend_comics('The Avengers')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index(['Marvel Universe Avengers Assemble: Civil War (2017) ',
       'Marvel Universe Avengers: Ultron Revolution Vol. 3 (Digest)',
       'Avengers: Endless Wartime (Hardcover)',
       'Avengers: Season One (Graphic Novel)', 'Avengers VS (2016) '],
      dtype='object', name='series')